Import librabries and read the three files into python as dataframes

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import time
from datetime import date
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LinearRegression as LinReg
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
import math
import plotly.graph_objects as go
import plotly.express as px
fifa=pd.read_csv('fifa21_train.csv')

# Cleaning data File1

In [2]:
def clean(file):
#-----------------------------------------STANDARDIZE HEADER NAMES------------------------------------------------------------#
    def headers(x):
        x.columns = x.columns.str.replace('[#,@,&,%,?,!,-,:]', '')
        x.columns = x.columns.str.replace(' ', '_')
        x.columns = x.columns.str.lower() 
        # Add the two numbers so we can later convert them into a string
        columns = ['ls','st','rs','lw','lf','cf','rf','rw','lam','cam','ram','lm','lcm','cm','rcm','rm','lwb','ldm', 'cdm','rdm','rwb','lb','lcb','cb','rcb','rb', 'gk']
        for col in columns:
            x[col] = x[col].str.split('+',n=1,expand = True)[0]
        return x
    # applying the function
    file = headers(file)
    pd.set_option('display.max_columns', None)
    file.head()
#-----------------------------------------REPLACE SPECIAL CHARACTERS AND CONVERTING UNITS-------------------------------------#
    # Remove the ' character
    file['height'] = file['height'].str.rstrip('"')
    # spliting the string into a list, converting it into an integer, and returning the dot product of the two arrays
    convertions_heights = [30.48, 2.54]
    file['height'] = file['height'].str.split("'").apply(pd.Series).astype(int).dot(convertions_heights)

#-----------------------------------------------------------------------------------------------------------------------------#
    # Removing lbs from the wheight
    file['weight'] = file['weight'].str.rstrip('lbs')
    file["weight"] = pd.to_numeric(file['weight'])
    file["weight"] = file["weight"].apply(lambda x: x*0.454) # ' lambda' to look up (Erin's tip)

#-----------------------------------------------------------------------------------------------------------------------------#
    # Removing the ★
    file['w/f']=file['w/f'].str.rstrip('★')
    file['sm']=file['sm'].str.rstrip('★')
    file['ir']=file['ir'].str.rstrip('★')
#-----------------------------------------------------------------------------------------------------------------------------#
    # Replace special characters for its actual meaning
    def clean_numbers(x):
        value=[]
        for i in x:
            i = i.replace("€", "")
            if 'K' in i:
                i = i.replace("K","")
                i = float(i)
                i = float(i*1000)
                i = int(i)
                value.append(i)
            elif 'M' in i:
                i = i.replace("M","")
                i = float(i)
                i = float(i*1000000)
                i = int(i)
                value.append(i)
            else:
                value.append(i)
        return value
    file['wage'] = clean_numbers(file['wage'])
    file['value'] = clean_numbers(file['value'])
    file['release_clause'] = clean_numbers(file['release_clause'])
    file['wage'] = pd.to_numeric(file['wage'])
    file['value'] = pd.to_numeric(file['value'])
    file['release_clause'] = pd.to_numeric(file['release_clause'])
    
#-----------------------------------------DEALING WITH NAN VALUES-------------------------------------------------------------#

    # Drop the column with really high NaN values
    file=file.drop(['loan_date_end'],axis=1)
    file.isna().sum()
#-----------------------------------------------------------------------------------------------------------------------------#
    '''In those columns with low NaN values that are relevant, fill with the mean.
    The Attacking Workrate and the Defensive Workrate columns are data of categorical nature, 
    therefore we encode the categorical variables'''
    file['composure'] = file['composure'].fillna(file['composure'].mean())
    file['a/w'] = file['a/w'].str.replace("High","3")
    file['a/w'] = file['a/w'].str.replace("Medium","2")
    file['a/w'] = file['a/w'].str.replace("Low","1")
    file['d/w'] = file['d/w'].str.replace("High","3")
    file['d/w'] = file['d/w'].str.replace("Medium","2")
    file['d/w'] = file['d/w'].str.replace("Low","1")
    file['a/w'] = file['d/w'].fillna(file['a/w'].median())
    file['d/w'] = file['d/w'].fillna(file['d/w'].median())
    file.isna().sum()
#-----------------------------------------------------------------------------------------------------------------------------#
    # In columns of law NaN, drop the raw where the NaN are presented
    '''Dealing with the NaN's of Position. The flexibility of a player to play in different positions is important enough to take into account, 
    therefore we encode the categorical variables into a column that contains in how many positions can he play despite his best position'''
    # Fill NaN values
    file['position']=file['position'].fillna('no_other')
    # Create a list of the other positions 
    other_positions=file['position'].str.split(' ')
    # Create a loop to enter the number of other positions and to take the best position out of that list
    size=[]
    for x in other_positions:
        z=len(x)-1
        size.append(z)
    file['other_p']=size
    # We can drop the column of the alternative positions. The positions are not relevant, but the number of different possitions is
    file=file.drop(columns='position',axis=1)
#-----------------------------------------------------------------------------------------------------------------------------#
    # Drop those columns that have low importance in relation with the target (OVA)
    file = file.drop(columns=['id','team__contract', 'contract', 'joined', 'name','nationality','hits','foot','bp'], axis=1)
    file = file.drop(columns=['club',], axis=1)
#-----------------------------------------------------------------------------------------------------------------------------#
    # Now I will change the dtype for all the objects remaining, excluding foot
    file[['w/f', 'sm', 'a/w', 'd/w', 'ir', 'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam','ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm','rwb','lb','lcb','cb','rcb','rb','gk']].apply(pd.to_numeric)
#-----------------------------------------------------------------------------------------------------------------------------#
    # Drop the rest of Nan's (non significant)
    file = file.dropna()
    return file

In [3]:
fifa = clean(fifa)

C:\Users\Jose María\AppData\Local\Temp\ipykernel_1500\920181195.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  x.columns = x.columns.str.replace('[#,@,&,%,?,!,-,:]', '')


In [4]:
fifa.head()

,age,height,weight,growth,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova,other_p
0,26,175.26,73.094,1,525000,4000,801000,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4,2,2,2,1,69,51,63,63,51,60,58,58,58,61,62,62,62,61,63,63,63,63,63,63,63,63,59,59,59,59,59,58,54,54,54,58,15,64,1
1,30,182.88,72.186,0,8500000,23000,0,365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3,4,1,1,2,83,75,68,82,33,71,77,77,77,77,77,77,77,77,76,76,76,76,68,68,68,76,57,53,53,53,57,53,48,48,48,53,18,77,2
2,33,162.56,60.836,0,9000000,49000,15300000,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4,4,2,2,2,80,77,78,86,27,56,73,73,73,80,79,79,79,80,80,80,80,79,74,74,74,79,59,56,56,56,59,53,41,41,41,53,12,80,1
3,22,177.80,69.008,13,275000,4000,694000,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2,2,2,2,1,57,44,54,57,57,60,50,50,50,51,51,51,51,51,53,53,53,53,56,56,56,53,56,58,58,58,56,57,58,58,58,57,14,59,1
4,23,180.34,68.100,8,725000,2000,1400000,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2,3,2,2,1,66,44,60,64,60,66,56,56,56,59,59,59,59,59,61,61,61,62,63,63,63,62,64,64,64,64,64,63,61,61,61,63,15,65,1


# Modelling

Spliting

In [5]:
y = fifa['ova'] #target, y lowercase
X = fifa.drop('ova', axis=1) #features, X uppercase

For scaling we will doing with normalization.
Normalizing data: make data range from 0 - 1, instead of from min to max

In [6]:
transf_mms = MinMaxScaler().fit(X)
X_nor = transf_mms.transform(X)
X_num_minmax = pd.DataFrame(X_nor, columns=X.columns)
X = X_num_minmax

Built the model

In [7]:
X_train, X_test, y_train, y_test=tts(X, y, test_size=.2)

In [8]:
# model
linreg=LinReg()    
# model training
linreg.fit(X_train, y_train)
# model prediction
y_pred_linreg=linreg.predict(X_test)   

Train/fit out model

In [9]:
lm = linear_model.LinearRegression() 
lm.fit(X,y)


LinearRegression()

Make predicion

In [10]:
pred = lm.predict(X_test)
pred

array([65.32324219, 81.91699219, 65.859375  , ..., 68.67675781,
       60.69726562, 64.98339844])

Evaluate results

In [11]:
rmse = np.sqrt(mse(linreg.predict(X_test), y_test))
rmse

2.195843850597733

# Cleaning data file2

In [16]:
fifa_v = pd.read_csv('fifa21_validate.csv')
fifa_v = clean(fifa_v)

C:\Users\Jose María\AppData\Local\Temp\ipykernel_1500\920181195.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  x.columns = x.columns.str.replace('[#,@,&,%,?,!,-,:]', '')


Spliting

In [17]:
y2 = fifa_v['ova'] #target, y lowercase
X2 = fifa_v.drop('ova', axis=1) #features, X uppercase

For scaling we will doing with normalization. Normalizing data: make data range from 0 - 1, instead of from min to max

In [18]:
X_nor = transf_mms.transform(X2)
X2 = pd.DataFrame(X_nor, columns=X2.columns)

Make predicions (using the model we built earlier)

In [19]:
validation_predictions = lm.predict(X2)
validation_predictions

array([65.24707031, 65.46875   , 53.91699219, ..., 72.89355469,
       65.01171875, 61.52734375])

Evaluate results

In [20]:
validations_mse= mse(validation_predictions, y2)
validations_mse

4.3498059948365055

In [21]:
rmse = np.sqrt(validations_mse)
rmse

2.085618851764748